In [ ]:
!pip install flopy

# MODFLOW 6.

## Componentes.

<img src="./figures/components_mf6.png">

Una simulación típicamente se basa en cuatro componentes:
* **Models**. Un modelo resuelve un proceso hidrológico; por ejemplo, el GWF model, resuelve la ecuación de flujo subterráneo usando el método CVFD.
* **Exchange**. Facilita la comunicación entre dos modelos; por ejemplo, un intercambio GWF-GWF, permite que algunas celdas de un modelo GWF estén hidráulicamente conectadas con celdas del otro modelo GWF.
* **Solutions**. Resuelve uno o más modelos hidrológicos, utiliza métodos iterativos para resolver sistemas no-lineales.
* **Timing**. Controla el paso de tiempo y determina el fin de una simulación.

**<font color='Green'>[1] Langevin, C.D., Hughes, J.D., Provost, A.M., Banta, E.R., Niswonger, R.G., and Panday, Sorab, 2017, Documentation for the MODFLOW 6 Groundwater Flow (GWF) Model: U.S. Geological Survey Techniques and Methods, book 6, chap. A55, 197 p., accessed August 4, 2017. </font>** https://doi.org/10.3133/tm6A55.

## Esquema para GWF.

<img src="./figures/modflow01.png" width=300px hspace="5" vspace="5" style="float: left;"/>

Taken from [4].

Figure 2. Schematic diagram showing several common ways in which the groundwater flow (GWF) and groundwatertransport (GWT) models can be configured within a simulation. The common use case depicted in (A) consists of a single    GWF model that is solved by a Numerical Solution. 


* The GWF Model described in [3] is divided into “packages,” as was done in previous MODFLOW versions.
* A package is the part of the model that deals with a single aspect of simulation.
* For example, the Well Package simulates the effect of wells, and the River Package simulates the effect of rivers.
* The GWF Model contains many packages and options that the user may or may not have occasion to use.


**<font color='Green'>[2] Langevin, C. D., Hughes, J. D., Provost, A. M., Russcher, M. J., & Panday, S. (2023). MODFLOW as a configurable Multi‐Model Hydrologic Simulator. Ground Water.</font>** https://doi.org/10.1111/gwat.13351

# Proceso de solución de flujo en 1D.

**Con base en**: 

**<font color='Green'>[3] MODFLOW 6 – Example problems, MODFLOW 6 Development Team, with contributions from Chieh Ying Chen and Mike Toews 02/07/2024.**</font>  **32 MOC3D Problem 1**. (Archivo: `mf6examples.pdf`).


## Paso 0. Importación de bibliotecas

In [1]:
import os   # Intefaces con el sistema operativo.
import numpy as np # Manejo de arreglos numéricos multidimensionales
import matplotlib.pyplot as plt # Graficación

# Biblioteca y módulos de flopy
import flopy
from flopy.plot.styles import styles

# Biblioteca con algunas funciones espacializadas para este proyeto
import vflow

## Paso 1. Definición de parámetros del problema.

### Parámetros para la discretización del espacio.

<img src="./figures/mesh_plainview_mf6.png" width=300px hspace="5" vspace="5" style="float: right;"/>
<img src="./figures/mesh_3D_mf6.png" width=300px hspace="5" vspace="5" style="float: right;"/>

* El modelo de malla consiste de 1 capa, 120 columnas y 1 renglón.
* La longitud del renglón es de 12 [cm].
* La longitud de la columna es 0.1 [cm].
* Con la información anterior se calcula el ancho del renglón, DELC, y de las columnas, DELR, que ambos casos debe ser 0.1 $cm$.
* La parte superior (TOP) de la celda es 1.0 [cm] y la parte inferior (BOTTOM) es cero.
* La longitud de la capa es igual a 1.0 [cm], valor que se calcula de |TOP - BOTTOM|.

<img src="./figures/flow_mf6.png">

|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Length of system (rows) |12.0| cm | `mesh.row_length` |
|Number of layers |1| | `mesh.nlay` |
|Number of rows |1| | `mesh.nrow` |
|Number of columns |120| | `mesh.ncol` |
|Column width |0.1| cm | `mesh.delr` |
|Row width |0.1| cm | `mesh.delc`|
|Top of the model |1.0| cm | `mesh.top`|
|Layer bottom elevation (cm) |0| cm | `mesh.bottom` |

In [2]:
mesh = vflow.MeshDis(
    nrow = 1,    # Number of rows
    ncol = 120,  # Number of columns
    nlay = 1,    # Number of layers
    row_length = 12.0,    # Length of system ($cm$)
    column_length = 0.1,  # Length of system ($cm$)
    top = 1.0,   # Top of the model ($cm$)
    bottom = 0,  # Layer bottom elevation ($cm$)
)

vflow.nice_print(mesh.get_dict(), 'Space discretization')


Space discretization
------------------------------
          row_length = 12.0      
          col_length = 0.1       
          lay_length = 1.0       
                ncol = 120       
                nrow = 1         
                nlay = 1         
                delr = 0.1       
                delc = 0.1       
                dell = 1.0       
                 top = 1.0       
              bottom = 0         


### Parámetros para la discretización del tiempo.

<center>
<img src="../figures/time_step.png" width=500px>
</center>
  
The length of a time step is calculated by multiplying the length of the previous time step by TSMULT. 
The length of the first time step, $\Delta t_1$, is related to PERLEN, NSTP, and TSMULT by the relation:
$$
\Delta t_1= \frac{\text{PERLEN}}{\text{NSTP}} \;\; \text{para} \;\; \text{TSMULT} = 1
$$

$$
\Delta t_1= \text{PERLEN} \frac{\text{TSMULT} - 1}{\text{TSMULT}^\text{nstp}-1} \;\; \text{para} \;\; \text{TSMULT} \neq 1
$$

The length of each successive time step is computed as

$$
\Delta t = \Delta t_{old} \text{TSMULT}
$$

where:
* `perlen` (double) is the length of a stress period.
* `nstp` (integer) is the number of time steps in a stress period.
* `tsmult` (double) is the multiplier for the length of successive time steps.
  
**<font color='Green'>[4] Hughes, J.D., Langevin, C.D., and Banta, E.R., 2017, *Documentation for the MODFLOW 6 framework: U.S. Geological Survey Techniques and Methods*, book 6, chap. A57, 40 p.,</font>** https://doi.org/10.3133/tm6A57. **Timing Module, pp 10,12**.

|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Number of stress periods |1| | `tm_par['nper']` |
|Total time |120| s | `tm_par['total_time']` |
|Number of time steps| 1 | | `tm_par['nstp']`|
|Multiplier | 1 | | `tm_par['tsmult']`|

In [ ]:
tm_par = dict(nper = 1,           # Number of periods
              total_time = 120.0, # Simulation time ($s$)
              nstp = 1.0,         # Number of time steps
              tsmult = 1.0        # Multiplier for the length of successive time steps.
              )

vflow.nice_print(tm_par, 'Time discretization')

### Parámetros físicos.

|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Specific discharge |0.1| cm s$^{-1}$ | `ph_par['specific_discharge']` |
|Hydraulic conductivity |0.01| cm s$^{-1}$ | `ph_par['hydraulic_conductivity']` |
|Source concentration |1.0| unitless | `ph_par['source_concentration']` |


In [ ]:
## Model Units
ml_units = {}
ml_units["time"] = "seconds"
ml_units["length"] = "centimeters"
vflow.nice_print(ml_units, 'Units')

## Physical parameters
ph_par = {}
ph_par["hydraulic_conductivity"] = 0.01  # Hydraulic conductivity ($cm s^{-1}$)
ph_par["specific_discharge"] = 0.1  # Specific discharge ($cm s^{-1}$)
ph_par["source_concentration"] = 1.0  # Source concentration (unitless)
vflow.nice_print(ph_par, 'Physical parameters')

## Paso 2. Ruta hacia el ejecutable de MODFLOW6 y nombre de la simulación.

In [ ]:
os_par = dict(ws = os.getcwd(), # Ruta de donde estamos actualmente
              mf6_exe = '/home/jovyan/GMMC/WMA/mf6/bin/mf6', # Ejecutable
              name = 'flow', # Nombre de la simulación
              head_file = "flow.hds", 
              budget_file = "flow.bud"
              )

In [ ]:
vflow.nice_print(os_par,'MODFLOW 6 + output files')

## Paso 3. Definición de la simulación (`MFSimulation` object)

**Recordemos que:**

<img src="./figures/modflow01.png" width=300px hspace="5" vspace="5" style="float: left;"/>

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name = os_par['name'], 
                             sim_ws   = os_par['ws'], 
                             exe_name = os_par['mf6_exe'])
print(sim)

## Paso 4. Definición de la discretización temporal (`ModflowTDis` object)

In [ ]:
flopy.mf6.ModflowTdis(sim, 
                      nper = tm_par["nper"], 
                      perioddata = ((tm_par["total_time"], 
                                     tm_par["nstp"], 
                                     tm_par["tsmult"]),), 
                      time_units = ml_units["time"])

## Paso 5. Definición de la solución numérica (`ModflowIms` object)

In [ ]:
ims = flopy.mf6.ModflowIms(sim)
print(ims)

## GWF: Paquetes

**<font color='Green'>Véase [1].</font>**

<img src="./figures/gwf_mf6.png" width=500px hspace="5" vspace="5" style="float: left;"/>
<img src="./figures/gwf_mf6_pack.png" width=500px hspace="5" vspace="5" style="float: left;"/>

## Paso 6. Modelo de Flujo (ModflowGwf object)

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, 
                           modelname  = os_par['name'], 
                           save_flows = True)
print(gwf)

## Paso 7. Paquete: discretización espacial (`ModflowGwfdis` object)

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf,
                              length_units = ml_units["length"],
                              nlay = mesh.nlay,
                              nrow = mesh.nrow,
                              ncol = mesh.ncol,
                              delr = mesh.delr,
                              delc = mesh.delc,
                              top  = mesh.top,
                              botm = mesh.bottom,
)
print(dis)

## Paso 8. Paquete: condiciones iniciales (`ModflowGwfic` object)

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, 
                            strt=1.0 # Initial head
                           ) 
print(ic)

## Paso 9. Paquete: propiedades de flujo en los nodos (`ModflowGwfnpd` object)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf,
                              save_specific_discharge = True,
                              save_saturation = True,
                              icelltype = 0,
                              k = ph_par['hydraulic_conductivity'],
)
print(npf)

## Paso 10. Paquete: CHD (`ModflowGwfchd` object)

In [ ]:
chd = flopy.mf6.ModflowGwfchd(gwf, 
                              stress_period_data=[[(0, 0, mesh.ncol - 1), # Node
                                                   1.0]]                  # Constant value
                             ) 
print(chd)

## Paso 11. Paquete: Pozos (`ModflowGwfwel` object)

In [ ]:
q   = ph_par["specific_discharge"] * mesh.delc * mesh.delr * mesh.top
aux = ph_par["source_concentration"]

wel = flopy.mf6.ModflowGwfwel(gwf,
                              stress_period_data = [[(0, 0, 0), q, aux,]],
                              pname = "WEL-1",
                              auxiliary = ["CONCENTRATION"],
)

print(wel)

## Paso 12. Paquete: salida (`ModflowGwfoc` object)

In [ ]:
oc = flopy.mf6.ModflowGwfoc(gwf,
                            head_filerecord   = os_par['head_file'],
                            budget_filerecord = os_par['budget_file'],
                            saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")],
)
print(oc)

## Paso 13. Escritura de los archivos de entrada para MODFLOW 6

In [ ]:
sim.write_simulation()

## Paso 14. Ejecución de la simulación.

In [ ]:
sim.run_simulation()

## Paso 15. Postprocessing

In [ ]:
# Obtenemos los resultados de la carga hidráulica
head = flopy.utils.HeadFile(os.path.join(os_par['ws'], os_par['head_file'])).get_data()

# Obtenemos los resultados del BUDGET
bud  = flopy.utils.CellBudgetFile(os.path.join(os_par['ws'], os_par['budget_file']),
                                  precision='double')
# Obtenemos las velocidades
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

In [ ]:
# Verificamos el tipo y dimensiones de los arreglos donde
# están almacenados la carga hidráulica, el BUDGET, y la velocidad.
print('Head : ', type(head), head.shape)
print('Budget : ', type(bud), bud.shape)
print('spdis : ', type(spdis), spdis.shape)
print('qx : ', type(qx), qx.shape)
print('qy : ', type(qy), qy.shape)
print('qz : ', type(qz), qz.shape)

In [ ]:
print(head.shape, '\n', head)

In [ ]:
print(qx.shape, '\n', qx)

In [ ]:
with styles.USGSPlot():
    plt.rcParams['font.family'] = 'DeJavu Sans'
    x, _, _ = mesh.get_coords()
    plt.figure(figsize=(10,3))
    plt.plot(x, head[0, 0], marker=".", ls ="-", mec="blue", mfc="none", markersize="1", label = 'Head')
    plt.xlim(0, 12)
    plt.xticks(ticks=np.linspace(0, mesh.row_length,13))
    plt.xlabel("Distance (cm)")
    plt.ylabel("Head")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plt.figure(figsize=(10,0.15))
ax = plt.gca()
#ax.set_aspect('equal')
pmv0 = flopy.plot.PlotMapView(gwf, ax=ax)
pmv0.plot_grid(colors='dimgray', lw=0.5)
plt.yticks(ticks=[0, 0.1],fontsize=8)

plt.show()


# 